In [35]:
import sys
import numpy as np
import pandas as pd
import cPickle
import multiprocessing
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

### Reading tweets

In [12]:
review_train = pd.read_csv('../data/kinopoisk_train.csv', encoding='utf-8')

In [13]:
review_train.head()

,text,label
0,"\n Прежде чем вообще что-то говорить, нуж...",1
1,"\nНа мой взгляд, любой боевик должен быть стил...",1
2,\nОб этом фильме мне довелось слышать не один ...,1
3,"\nОчень сложно вспомнить фильм, в котором не б...",0
4,\nВ фильме показан нетрадиционный Шерлок Холмс...,1


In [14]:
texts, labels = review_train.text.values, review_train.label.values

### Reading vocabulary and embeddings

In [15]:
word2id, embeddings = cPickle.load(open('../data/w2v/vectors_l.pkl', 'rb'))
# word2id, embeddings = cPickle.load(open('../data/w2v/parkin_vectors.pkl', 'rb'))

In [16]:
vocabulary = word2id.keys()
eos_id = word2id[u'</s>']

### Lemmatizing and replacing words with ids

In [17]:
from nltk.tokenize import RegexpTokenizer
import pymorphy2

tokenizer = RegexpTokenizer(u'[а-яА-Яa-zA-Z]+')
morph = pymorphy2.MorphAnalyzer()

def text2seq(text):
    tokens_norm = [morph.parse(w)[0].normal_form for w in tokenizer.tokenize(text)]
    return [word2id[w] for w in tokens_norm if w in vocabulary] + [eos_id]

sample = texts[49]

print sample
print u' '.join(tokenizer.tokenize(sample))
print u' '.join([morph.parse(w)[0].normal_form for w in tokenizer.tokenize(sample)])
print text2seq(sample)


Мне странно, что отзывы о таком шедевре настолько холодные… Такое впечатление что тут открыли форум критиков «ВК»… 

Фильм поражает своей масштабностью, красотой и глубиной образов и самого мира — до сих пор удивляюсь, как мог один человек создать целый мир, целую эпоху со своими мифами, героями, географией, летоисчеслением, прошлым, настояшим и будующим! 

Гений Толкиена идеально перенесен на большие экраны и экранизация романа просто разрывает тебя на куски! Когда смотришь фильм — ты попадаешь в сказку… причем не добрую и поучительную, а реалистичную и жесткую, где высоко приподнесены такие извечные ценности как честь, достоинство, отвага, долг. 

А музыка фильма это отдельный разговор! Музыкальная тема фильма выше всяких похвал — ну как могут не тронуть сердце эльфийские баллады или героические мотивы на фоне батальных сцен!

Бесспорно каждый Оскар из 11 полученных взят заслужено!

10 из 10
Мне странно что отзывы о таком шедевре настолько холодные Такое впечатление что тут открыли 

Dumping

Loading

In [18]:
X = cPickle.load(open('../data/X_kinopoisk_train.pkl', 'rb'))

Drop samples with the length > 150

In [19]:
length_max = 1024
y = review_train.label.values
y = y[np.array(map(len, X)) < length_max]
X = [x for x in X if len(x) < length_max]

### Zero padding

In [20]:
X = [x + [eos_id]*(length_max - len(x)) for x in X]

### Examples

In [21]:
print X[0]

[799, 64, 102, 4, 12, 73, 137, 2096, 281, 67, 736, 2359, 4, 7, 663, 31, 57, 94, 139, 11, 51, 8137, 281, 506, 10224, 1, 9144, 8137, 16, 6134, 605, 113, 3, 2, 1330, 506, 14, 1236, 4, 80, 12, 1, 198, 36, 48, 31095, 1, 8400, 6981, 6, 47766, 4082, 2, 1714, 12, 103, 20511, 3, 9, 2205, 15693, 15, 3, 478, 644, 64, 8400, 17, 131, 46, 47766, 11199, 140, 2205, 8687, 3, 565, 71, 39, 31, 178, 506, 6, 26, 50, 8687, 78, 73, 36, 16, 1600, 9748, 5, 2375, 1, 60, 2, 39, 485, 135, 1416, 1403, 6888, 9294, 5, 4, 9294, 5, 30, 38, 4740, 335, 33, 67, 17, 57, 1, 49, 9294, 4, 38, 561, 9294, 5, 9294, 7, 48, 3, 2, 50, 1330, 45, 1, 14, 65, 61240, 1525, 2, 30, 5352, 50, 244, 69, 65, 335, 26177, 15630, 24, 4619, 263, 17, 88, 192, 76, 170, 191, 9, 256, 4, 12, 6401, 8, 69, 368, 9, 732, 29, 1394, 43, 3743, 2, 8882, 427, 109, 2345, 5, 39, 89, 40, 123, 4, 10, 3, 256, 107, 5068, 182, 1, 910, 6, 4211, 78, 1, 281, 1223, 36, 76, 344, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### Split into train and validation sets

In [22]:
X = np.array(X)

In [23]:
def cls2probs(cls):
    if cls == -1:
        return [1.,0., 0.]
    if cls == 0:
        return [0.,1., 0.]
    else:
        return [0.,0.,1.]
y = np.array([cls2probs(cls) for cls in y])

In [24]:
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.1

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=40)

In [27]:
print "Train class frequencies:\t", [col.nonzero()[0].shape[0] for col in y_tr.transpose()]
print "Validation class frequencies:\t", [col.nonzero()[0].shape[0] for col in y_val.transpose()]
print "Constant classifier's validation accuracy:\t", [col.nonzero()[0].shape[0] for col in y_val.transpose()][2] * 1. / y_val.shape[0]

Train class frequencies:	[3385, 3237, 19514]
Validation class frequencies:	[367, 387, 2151]
Constant classifier's validation accuracy:	0.740447504303


### Resampling

# Network learning

In [28]:
import tensorflow as tf
from tensorflow.contrib.rnn import GRUCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from tensorflow.contrib.layers import fully_connected

from utils import *

In [31]:
from sklearn.metrics import f1_score
f_macro = lambda y1, y2: f1_score(y1, y2, average="macro")
f_micro = lambda y1, y2: f1_score(y1, y2, average="micro")

y_pred_major = np.zeros(y_val.shape)
y_pred_major[:,2] = 1.
print "Constant classifier's macro-averaged F-score on validation set:", f_macro(y_val, y_pred_major)
print "Constant classifier's micro-averaged F-score on validation set:", f_micro(y_val, y_pred_major)

Constant classifier's macro-averaged F-score on validation set: 0.283623417722
Constant classifier's micro-averaged F-score on validation set: 0.740447504303


### Bi-RNN with Attention mechanism

Attention mechanism formulas and notation
$$
\upsilon_{t}=\tanh{(W_{\omega}\left[\overrightarrow{h_{t}},\overleftarrow{h_{t}}\right]+b_{\omega})}\\
\alpha_{t}=\frac{\exp{(\upsilon_{t}^{T}u_{\omega})}}{\sum_{j=1}^{T}\exp{(\upsilon_{j}^{T}u_{\omega})}}\\
\upsilon=\sum_{t=1}^{T}\alpha_{t}h_{t}	
$$

In [32]:
EMBED_DIM = 300
SEQ_LENGTH = length_max
HIDDEN_SIZE = 64
ATTENTION_SIZE = 32
NUM_CLASSES = 3

tf.reset_default_graph()

batch_ph   = tf.placeholder(tf.int32, [None, SEQ_LENGTH])
target_ph  = tf.placeholder(tf.float32, [None, NUM_CLASSES])
seq_len_ph = tf.placeholder(tf.int32, [None])
keep_prob_ph = tf.placeholder(tf.float32)

embeddings_ph = tf.placeholder(tf.float32, [len(vocabulary), EMBED_DIM])
embeddings_var = tf.Variable(tf.constant(0., shape=[len(vocabulary), EMBED_DIM]), trainable=False)
init_embeddings = embeddings_var.assign(embeddings_ph)

batch_embedded = tf.nn.embedding_lookup(embeddings_var, batch_ph)

# Bi-RNN layers
outputs,_ = bi_rnn(GRUCell(HIDDEN_SIZE), GRUCell(HIDDEN_SIZE),
                   inputs=batch_embedded,sequence_length=seq_len_ph, dtype=tf.float32, scope="bi_rnn1")  
outputs = tf.concat(outputs, 2)
# outputs,_ = bi_rnn(GRUCell(HIDDEN_SIZE), GRUCell(HIDDEN_SIZE),
#                          inputs=outputs,sequence_length=seq_len_ph, dtype=tf.float32, scope="bi_rnn2")
# outputs = tf.concat(outputs, 2)

# Attention mechanism
W_omega = tf.Variable(tf.random_normal([2 * HIDDEN_SIZE, ATTENTION_SIZE], stddev=0.1))
b_omega = tf.Variable(tf.random_normal([ATTENTION_SIZE], stddev=0.1))
u_omega = tf.Variable(tf.random_normal([ATTENTION_SIZE], stddev=0.1))

v = tf.tanh(tf.matmul(tf.reshape(outputs, [-1, 2 * HIDDEN_SIZE]), W_omega) + tf.reshape(b_omega, [1, -1]))
vu = tf.matmul(v, tf.reshape(u_omega, [-1, 1]))
exps = tf.reshape(tf.exp(vu), [-1, SEQ_LENGTH])
alphas = exps / tf.reshape(tf.reduce_sum(exps, 1), [-1, 1])

# Output of Bi-RNN reduced with attention vector
output = tf.reduce_sum(outputs * tf.reshape(alphas, [-1, SEQ_LENGTH, 1]), 1)

# Dropout
drop = tf.nn.dropout(output, keep_prob_ph)

# Fully connected layer
W = tf.Variable(tf.truncated_normal([HIDDEN_SIZE * 2, NUM_CLASSES], stddev=0.1), name="W")
b = tf.Variable(tf.constant(0., shape=[NUM_CLASSES]), name="b")
y_hat = tf.nn.xw_plus_b(drop, W, b, name="scores")

In [33]:
# Adam parameters
LEARNING_RATE = 0.008
EPSILON = 1e-5
BETA1 = 0.9
BETA2 = 0.9
# L2 regularization coefficient
BETA = 0

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_ph, logits=y_hat), name="cross_entropy")
l2_loss = tf.nn.l2_loss(W, name="l2_loss")
loss = cross_entropy + l2_loss * BETA
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE, beta1=BETA1, beta2=BETA2,
                                   epsilon=EPSILON).minimize(loss)
# optimizer = tf.train.AdadeltaOptimizer(learning_rate=LEARNING_RATE, rho=0.7).minimize(loss)
# optimizer = tf.train.AdagradOptimizer(learning_rate=LEARNING_RATE, initial_accumulator_value=0.1).minimize(loss)
# optimizer = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE, momentum=0.1).minimize(loss)
# optimizer = tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE, decay=0.9, momentum=0.1).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(target_ph, 1), tf.argmax(y_hat, 1)), tf.float32))

In [36]:
DROPOUT = 0.8  # Probability of keeping a neuron
BATCH_SIZE = 256
EPOCHS = 10

train_batch_generator = batch_generator(X_tr, y_tr, BATCH_SIZE)

loss_tr_l = []
loss_val_l = []
ce_tr_l = []  # Cross-entropy
ce_val_l = []
acc_tr_l = []  # Accuracy
acc_val_l = []
f_macro_tr_l = []
f_macro_val_l = []
f_fair_tr_l = []
f_fair_val_l = []
max_val_f = 0.

with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(init_embeddings, feed_dict={embeddings_ph: embeddings})
    print "Start learning..."
    for epoch in range(EPOCHS):
        for i in range(int(X_tr.shape[0] / BATCH_SIZE)):
            x_batch, y_batch = train_batch_generator.next()
            seq_len_tr = np.array([list(x).index(eos_id) + 1 for x in x_batch])
            sess.run(optimizer, feed_dict={batch_ph: x_batch, target_ph: y_batch,
                                           seq_len_ph: seq_len_tr, keep_prob_ph: DROPOUT})

        y_pred_tr, ce_tr, loss_tr, acc_tr = sess.run([y_hat, cross_entropy, loss, accuracy],
                                                     feed_dict={batch_ph: x_batch, target_ph: y_batch,
                                                                seq_len_ph: seq_len_tr, keep_prob_ph: 1.0})

        y_pred_val, ce_val, loss_val, acc_val = [], 0, 0, 0
        num_val_batches = X_val.shape[0] / BATCH_SIZE
        for i in range(num_val_batches):
            x_batch_val, y_batch_val = X_val[i * BATCH_SIZE: (i + 1) * BATCH_SIZE], y_val[i * BATCH_SIZE: (
                                                                                                              i + 1) * BATCH_SIZE]
            seq_len_val = np.array([list(x).index(eos_id) + 1 for x in x_batch_val])
            y_pred_val_, ce_val_, loss_val_, acc_val_ = sess.run([y_hat, cross_entropy, loss, accuracy],
                                                                 feed_dict={batch_ph: x_batch_val,
                                                                            target_ph: y_batch_val,
                                                                            seq_len_ph: seq_len_val,
                                                                            keep_prob_ph: 1.0})
            y_pred_val += list(y_pred_val_)
            ce_val += ce_val_
            loss_val += loss_val_
            acc_val += acc_val_

        y_pred_val = np.array(y_pred_val)
        ce_val /= num_val_batches
        loss_val /= num_val_batches
        acc_val /= num_val_batches

        y_pred_tr = np.array([cls2probs(cls) for cls in np.argmax(y_pred_tr, 1) - 1])
        y_pred_val = np.array([cls2probs(cls) for cls in np.argmax(y_pred_val, 1) - 1])
        f_macro_tr, f_micro_tr = f_macro(y_batch, y_pred_tr), f_micro(y_batch, y_pred_tr)
        f_macro_val, f_micro_val = f_macro(y_val[:num_val_batches * BATCH_SIZE], y_pred_val), f_micro(
            y_val[:num_val_batches * BATCH_SIZE], y_pred_val)

        loss_tr_l.append(loss_tr)
        loss_val_l.append(loss_val)
        ce_tr_l.append(ce_tr)
        ce_val_l.append(ce_val)
        acc_tr_l.append(acc_tr)
        acc_val_l.append(acc_val)
        f_macro_tr_l.append(f_macro_tr)
        f_macro_val_l.append(f_macro_val)

        print "epoch: {}".format(epoch)
        print "\t Train loss: {:.3f}\t ce: {:.3f}\t acc: {:.3f}\t f_macro: {:.3f}".format(
            loss_tr, ce_tr, acc_tr, f_macro_tr)
        print "\t Valid loss: {:.3f}\t ce: {:.3f}\t acc: {:.3f}\t f_macro: {:.3f}".format(
            loss_val, ce_val, acc_val, f_macro_val)

        if f_macro_val > max_val_f:
            max_val_f = f_macro_val
            saver.save(sess, 'model')

    results = [max(acc_val_l), max(f_macro_val_l)]

Start learning...-f
epoch: 0
	 Train loss: 0.568	 ce: 0.568	 acc: 0.773	 f_macro: 0.490
	 Valid loss: 0.502	 ce: 0.502	 acc: 0.809	 f_macro: 0.521
epoch: 1
	 Train loss: 0.508	 ce: 0.508	 acc: 0.812	 f_macro: 0.583
	 Valid loss: 0.450	 ce: 0.450	 acc: 0.825	 f_macro: 0.600
epoch: 2
	 Train loss: 0.408	 ce: 0.408	 acc: 0.836	 f_macro: 0.605
	 Valid loss: 0.447	 ce: 0.447	 acc: 0.829	 f_macro: 0.586
epoch: 3
	 Train loss: 0.445	 ce: 0.445	 acc: 0.828	 f_macro: 0.636
	 Valid loss: 0.425	 ce: 0.425	 acc: 0.837	 f_macro: 0.633
epoch: 4
	 Train loss: 0.439	 ce: 0.439	 acc: 0.832	 f_macro: 0.577
	 Valid loss: 0.444	 ce: 0.444	 acc: 0.836	 f_macro: 0.600
epoch: 5
	 Train loss: 0.338	 ce: 0.338	 acc: 0.848	 f_macro: 0.653
	 Valid loss: 0.437	 ce: 0.437	 acc: 0.836	 f_macro: 0.658
epoch: 6
	 Train loss: 0.285	 ce: 0.285	 acc: 0.891	 f_macro: 0.736
	 Valid loss: 0.473	 ce: 0.473	 acc: 0.836	 f_macro: 0.639
epoch: 7
	 Train loss: 0.244	 ce: 0.244	 acc: 0.902	 f_macro: 0.796
	 Valid loss: 0.501	 ce

In [37]:
results

[0.83700284090909094, 0.67486937331503383]

---

---

---

# Testing

In [38]:
review_test = pd.read_csv('../data/kinopoisk_test.csv', sep=',', encoding='utf-8')
texts_test, labels_test = review_test.text.values, review_test.label.values

In [39]:
X_test = cPickle.load(open('../data/X_kinopoisk_test.pkl', 'rb'))

In [40]:
X_test = [x[:length_max - 1] for x in X_test]
X_test = [x + [eos_id]*(length_max - len(x)) for x in X_test]
X_test = np.array(X_test)
y_test = np.array([cls2probs(cls) for cls in labels_test])

In [41]:
X_test.shape, y_test.shape

((7270, 1024), (7270, 3))

In [144]:
print "Constant classifier's test accuracy:\t", [col.nonzero()[0].shape[0] for col in y_test.transpose()][2] * 1. / y_test.shape[0]
y_pred_major = np.zeros(y_test.shape)
y_pred_major[:,2] = 1.
print "Constant classifier's macro-averaged F-score on validation set:", f_macro(y_test, y_pred_major)

Constant classifier's test accuracy:	0.735763411279
Constant classifier's macro-averaged F-score on validation set: 0.282589745622


In [42]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('model.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./'))

    y_pred_test, ce_test, loss_test, acc_test = [], 0, 0, 0
    num_test_batches = X_test.shape[0] / BATCH_SIZE
    for i in range(num_test_batches):
        x_batch_test, y_batch_test = X_test[i * BATCH_SIZE : (i + 1) * BATCH_SIZE],\
                                   y_test[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]
        seq_len_test = np.array([list(x).index(eos_id) + 1 for x in x_batch_test])
        y_pred_test_, ce_test_, loss_test_, acc_test_ = sess.run([y_hat, cross_entropy, loss, accuracy],
                                                     feed_dict={batch_ph: x_batch_test, target_ph: y_batch_test,
                                                                seq_len_ph: seq_len_test, keep_prob_ph: 1.0})
        y_pred_test += list(y_pred_test_)
        ce_test += ce_test_
        loss_test += loss_test_
        acc_test += acc_test_

    y_pred_test = np.array(y_pred_test)
    ce_test /= num_test_batches
    loss_test /= num_test_batches
    acc_test /= num_test_batches

INFO:tensorflow:Restoring parameters from ./model


In [43]:
y_pred_test = np.array([cls2probs(cls) for cls in np.argmax(y_pred_test, 1) - 1])
f_macro_test = f_macro(y_test[:num_test_batches * BATCH_SIZE], y_pred_test)
print acc_test, f_macro_test

0.811941964286 0.637816372677


---

## Attention weights

In [44]:
id2word = {}
for word, word_id in word2id.iteritems():
    id2word[word_id] = word

In [113]:
print review_test.text.values[22]


Нолан — мой любимый режиссер. А его «Помни» — мой любимый фильм. Но только сейчас я с удивлением узнала, что «Престиж» снял тот же режиссер. Собственно, сам фильм я посмотрела не из-за громких имен актеров и режиссера, а потому, что прочитала книгу. Произведение Кристофера Приста словно схватило меня за душу и не отпускало до самого конца. А вечером я пересказывала историю двух иллюзионистов своей подруге, которая понимала меня с трудом, ведь сам сюжет настолько лихо закручен, а конец просто шедеврален! И на этой позитивной ноте я взялась за фильм. Скажу честно — разочаровал. Скучнее я ничего не видела. После нескольких первых минут я начала «пропадать» в сюжете. Он оказался уж больно запутанным, да так, что, кажется, сценарист сам в нем запутался с режиссером на пару. Всё время пыталась связать происходящее на экране с книгой, но, как ни старалась, ничего общего не нашла. Вкуснейшая интрига Приста была отвратительно чем-то разбавлена. Блюдо остыло и дорога ему на помойку. Единственно

In [ ]:
# sent = review_test.text.values[22]
# print sent
sent = """Нолан— мой любимый режиссер. А его «Помни»— мой любимый фильм. Но только сейчас я с удивлением узнала, что «Престиж» снял тот же режиссер. Собственно, сам фильм я посмотрела не из -за громких имен актеров и режиссера, а потому, что прочитала книгу. Произведение Кристофера Приста словно схватило меня за душу и не отпускало до самого конца. А вечером я пересказывала историю двух иллюзионистов своей подруге, которая понимала меня с трудом, ведь сам сюжет настолько лихо закручен, а конец просто шедеврален! И на этой позитивной ноте я взялась за фильм. Скажу честно— разочаровал. Скучнее я ничего не видела. После нескольких первых минут я начала «пропадать» в сюжете. Он оказался уж больно запутанным, да так, что, кажется, сценарист сам в нем запутался с режиссером на пару. Всё время пыталась связать происходящее на экране с книгой, но, как ни старалась, ничего общего не нашла. Вкуснейшая интрига Приста была отвратительно чем- то разбавлена. Блюдо остыло и дорога ему на помойку. Единственное, что хоть немного меня порадовало, так это актерский состав. Игра потрясающая, но со Скарлетт что- то было не так. Возможно, дело в самой Оливии. Кажется, в книге этот персонаж был полнее. 

В общем, в попытке переделать роман на большой экран, была уничтожена вся острота начинки. Для меня, это— худшая экранизация прочитанной книги.

Вот так вот,"""

In [129]:
i = 0
for q, r in zip(sent.split(), map(id2word.get, X_test[22][:200])):
    print i, q, r
    i += 1

0 Нола— нолана
1 мой мой
2 любимый любимый
3 режиссер. режиссёр
4 А а
5 его он
6 «Помни»— помнить
7 мой мой
8 любимый любимый
9 фильм. фильм
10 Но но
11 только только
12 сейчас сейчас
13 я я
14 с с
15 удивлением удивление
16 узнала, узнать
17 что что
18 «Престиж» престиж
19 снял снять
20 тот тот
21 же же
22 режиссер. режиссёр
23 Собственно, собственно
24 сам сам
25 фильм фильм
26 я я
27 посмотрела посмотреть
28 не не
29 из из
30 -за за
31 громких громкий
32 имен имя
33 актеров актёр
34 и и
35 режиссера, режиссёр
36 а а
37 потому, потому
38 что что
39 прочитала прочитать
40 книгу. книга
41 Произведение произведение
42 Кристофера кристофер
43 Приста пристый
44 словно словно
45 схватило схватить
46 меня я
47 за за
48 душу душить
49 и и
50 не не
51 отпускало отпускать
52 до до
53 самого самый
54 конца. конец
55 А а
56 вечером вечером
57 я я
58 пересказывала пересказывать
59 историю история
60 двух два
61 иллюзионистов иллюзионист
62 своей свой
63 подруге, подруга
64 которая который
65 пони

In [139]:
w = alphas_test[0]
w /= alphas_test.max() / 100
for word, coef in zip(sent.split(), w):
    print "\colorbox{yellow!%d}{%s}" % (int(coef), word)

\colorbox{yellow!4}{Нола—}
\colorbox{yellow!6}{мой}
\colorbox{yellow!6}{любимый}
\colorbox{yellow!8}{режиссер.}
\colorbox{yellow!3}{А}
\colorbox{yellow!4}{его}
\colorbox{yellow!3}{«Помни»—}
\colorbox{yellow!4}{мой}
\colorbox{yellow!4}{любимый}
\colorbox{yellow!4}{фильм.}
\colorbox{yellow!2}{Но}
\colorbox{yellow!1}{только}
\colorbox{yellow!0}{сейчас}
\colorbox{yellow!0}{я}
\colorbox{yellow!1}{с}
\colorbox{yellow!3}{удивлением}
\colorbox{yellow!2}{узнала,}
\colorbox{yellow!2}{что}
\colorbox{yellow!4}{«Престиж»}
\colorbox{yellow!1}{снял}
\colorbox{yellow!2}{тот}
\colorbox{yellow!2}{же}
\colorbox{yellow!7}{режиссер.}
\colorbox{yellow!5}{Собственно,}
\colorbox{yellow!6}{сам}
\colorbox{yellow!11}{фильм}
\colorbox{yellow!2}{я}
\colorbox{yellow!0}{посмотрела}
\colorbox{yellow!2}{не}
\colorbox{yellow!2}{из}
\colorbox{yellow!4}{-за}
\colorbox{yellow!3}{громких}
\colorbox{yellow!3}{имен}
\colorbox{yellow!5}{актеров}
\colorbox{yellow!2}{и}
\colorbox{yellow!6}{режиссера,}
\colorbox{yellow!3}{а}
\co

In [102]:
print " ".join(map(id2word.get, X_test[20][:200]))

сказать что я ждать этот фильм значит ничто не сказать скорсез один из тот режиссёр фильм который я ждать действительно искренне остров проклятый этакий жгучий коктейль молотов фильм без начало и без конец фильм яркий и сочный тенденция к накручивание сюжет быть уже в отступник а здесь он превзойти себя да и другой в жанр психологический триллер настолько гармонично выглядеть весь деталь и нюанс фильм что кажется что ты что то упустить что ты слабый чем фильм а фильм очень сильный музыкальный сопровождение на высокий уровень в прочее как всегда у скорсез ди каприо жечь весь фильм один коронный взгляд и бровь леонардо окупать билет с лихва сам сюжет не новый но бесспорный в исполнение скорсез тянуть на из антураж вышибать мозг на один минута в середина фильм переставать понимать где грань между реальность и вымысел к конец фильм переставать понимать а быть ли реальность реальность а бред бред за это фильм запросто можно назвать один из хороший работа в кинематограф за последний год из <

In [66]:
print review_test.text.values[13]


Фильм вне категорий, не потому что он так хорошо снят или там шикарная актерская игра. Все это там есть, но не это заставляет многих плакать… И не нарочитая слезодавильность. Заставляет очевидная реальность и простота чувств. Я задумался, а почему же слезы у всех? Никто не сказал, что ему жалко Хатико, никто не сказал, что так несправедлива жизнь. Нет в этой истории и нелепых случайностей, что заставляют так сопереживать. 

Мы плачем потому что понимаем, что мы сами никогда не сможем понять и испытать то, что чувствует Хати. И также полюбить…


In [81]:
print " ".join(map(id2word.get, X_test[13][:200]))

фильм вне категория не потому что он так хорошо снятой или там шикарный актёрский игра весь это там есть но не это заставлять многий плакать и не нарочитый заставлять очевидный реальность и простота чувство я задуматься а почему же слеза у весь никто не сказать что он жалко хатико никто не сказать что так несправедливый жизнь нет в этот история и нелепый случайность что заставлять так сопереживать мы плач потому что понимать что мы сам никогда не смочь понять и испытать то что чувствовать хатить и также полюбить </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [87]:
sentence = u"""Фильм вне категорий, не потому что он так хорошо снят или там шикарная актерская игра. Все это там есть, но не это заставляет многих плакать… И не нарочитая""" +\
u""" Заставляет очевидная реальность и простота чувств. Я задумался, а почему же слезы у всех? Никто не сказал, что ему жалко Хатико, никто не сказал, что так несправедлива жизнь. Нет в этой истории и нелепых случайностей, что заставляют так сопереживать. 

Мы плачем потому что понимаем, что мы сами никогда не сможем понять и испытать то, что чувствует Хати. И также полюбить…"""
print sentence

Фильм вне категорий, не потому что он так хорошо снят или там шикарная актерская игра. Все это там есть, но не это заставляет многих плакать… И не нарочитая Заставляет очевидная реальность и простота чувств. Я задумался, а почему же слезы у всех? Никто не сказал, что ему жалко Хатико, никто не сказал, что так несправедлива жизнь. Нет в этой истории и нелепых случайностей, что заставляют так сопереживать. 

Мы плачем потому что понимаем, что мы сами никогда не сможем понять и испытать то, что чувствует Хати. И также полюбить…


In [131]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('model_att.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./'))
    x_batch_test, y_batch_test = X_test[22:23], y_test[22:23]
    seq_len_test = np.array([list(x).index(eos_id) + 1 for x in x_batch_test])
    y_pred_test, alphas_test = sess.run([y_hat, alphas],
                                                 feed_dict={batch_ph: x_batch_test, target_ph: y_batch_test,
                                                            seq_len_ph: seq_len_test, keep_prob_ph: 1.0})

INFO:tensorflow:Restoring parameters from ./model


INFO:tensorflow:Restoring parameters from ./model


In [132]:
y_pred_test

array([[ 4.67509794,  1.55498695, -5.07856321]], dtype=float32)

In [88]:
for word, coef in zip(sentence.split()[:150], alphas_test[0,:150] * 1000 / 1.7):
    print "\colorbox{yellow!%d}{%s}" % (int(coef), word)

\colorbox{yellow!7}{Фильм}
\colorbox{yellow!4}{вне}
\colorbox{yellow!6}{категорий,}
\colorbox{yellow!4}{не}
\colorbox{yellow!6}{потому}
\colorbox{yellow!4}{что}
\colorbox{yellow!5}{он}
\colorbox{yellow!9}{так}
\colorbox{yellow!11}{хорошо}
\colorbox{yellow!8}{снят}
\colorbox{yellow!3}{или}
\colorbox{yellow!2}{там}
\colorbox{yellow!4}{шикарная}
\colorbox{yellow!12}{актерская}
\colorbox{yellow!4}{игра.}
\colorbox{yellow!3}{Все}
\colorbox{yellow!1}{это}
\colorbox{yellow!1}{там}
\colorbox{yellow!1}{есть,}
\colorbox{yellow!1}{но}
\colorbox{yellow!2}{не}
\colorbox{yellow!2}{это}
\colorbox{yellow!2}{заставляет}
\colorbox{yellow!2}{многих}
\colorbox{yellow!3}{плакать…}
\colorbox{yellow!2}{И}
\colorbox{yellow!3}{не}
\colorbox{yellow!12}{нарочитая}
\colorbox{yellow!11}{Заставляет}
\colorbox{yellow!38}{очевидная}
\colorbox{yellow!37}{реальность}
\colorbox{yellow!9}{и}
\colorbox{yellow!12}{простота}
\colorbox{yellow!24}{чувств.}
\colorbox{yellow!6}{Я}
\colorbox{yellow!2}{задумался,}
\colorbox{yello